In [14]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import pandas as pd
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from neo4j import  Driver
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0)

In [21]:
# Load your P&L table (CSV)
df = pd.read_csv("financial_statement_comparison.csv")  # Replace with your actual file
table_text = df.to_markdown(index=False)

# Prompt Template
prompt = ChatPromptTemplate.from_template("""
You are a financial analyst. Given the following profit and loss table, write a detailed but concise summary of the company's financial performance.
Use financial terms like revenue, gross profit, net income, and margins if applicable.

Table:
{table}

Summary:
""")

# Bind model and prompt
chain = prompt | llm | StrOutputParser()

# Generate summary
summary = chain.invoke({"table": table_text})
# Define the path to your existing text file
file_path = "finances.txt"

# Open the file in append mode and write the summary
with open(file_path, "a", encoding="utf-8") as file:
    file.write("\n" + summary)

print("Summary appended successfully.")
print(summary)


Summary appended successfully.
Based on the provided profit and loss table, here is a detailed but concise summary of the company's financial performance:

**Revenue and Gross Profit:**
The company's revenue has increased by 43% from March 31, 2023 to March 31, 2024, driven by growth in investments, trade receivables, and unbilled revenue. Gross profit has also increased, but the gross margin has declined slightly from 24.5% to 23.2% due to higher costs.

**Operating Performance:**
Operating expenses have increased by 35% primarily due to higher salaries, wages, and other expenses. As a result, operating profit has decreased by 12%. The operating margin has also declined from 15.6% to 13.5%.

**Net Income:**
Net income has increased by 18% to Rs. 178,422 due to a significant improvement in non-operating income, mainly driven by investments. The net margin has expanded from 10.1% to 11.3%.

**Cash Flow:**
The company's cash and cash equivalents have decreased by 22% from March 31, 2023 

In [19]:
load_dotenv()
#print(os.environ["NEO4J_URI"])
graph = Neo4jGraph(
    url="bolt://localhost:7688",
    username="neo4j",
    password="F9gsV3zmLpQ2xT"
)


/var/folders/06/bmb8g_1d55q5lgskjgst444h0000gn/T/ipykernel_22584/135865777.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [3]:
loader = TextLoader(file_path='finances.txt')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)


In [16]:

llm_transformer = LLMGraphTransformer(llm=llm)

# Initialize the list to store successfully converted graph documents
graph_documents = []

# Initialize a counter to track failed conversions
failed_conversions = 0

# Initialize a list to keep track of failed documents (optional)
failed_documents = []

# Iterate over each document in the list
for idx, document in enumerate(documents):
    print(f"""Document ${idx} processing. \n""")
    try:
        # Try converting the document to a graph document
        graph_document = llm_transformer.convert_to_graph_documents([document])
        
        # If conversion is successful, append to the result list
        graph_documents.append(graph_document)
    except Exception as e:
        print("failed\n")
        # If conversion fails, count the failure and store the document (optional)
        failed_conversions += 1
        failed_documents.append((idx, document, str(e)))  # Store index, document, and error message

# After the loop, print or log the results
print(f"Total documents: {len(documents)}")
print(f"Successfully converted: {len(graph_documents)}")
print(f"Failed conversions: {failed_conversions}")

# Optionally, you can log or handle the failed documents separately
if failed_conversions > 0:
    print("Failed Documents:")
    for failed in failed_documents:
        print(f"Document {failed[0]} failed due to error: {failed[2]}")

# Now you have 'graph_documents' with the successful conversions
# and 'failed_conversions' with the count of failed attempts.



Document $0 processing. 

Document $1 processing. 

Document $2 processing. 

Document $3 processing. 

Document $4 processing. 

Document $5 processing. 

Document $6 processing. 

Document $7 processing. 

Document $8 processing. 

Document $9 processing. 

Document $10 processing. 

Document $11 processing. 

Document $12 processing. 

Document $13 processing. 

Document $14 processing. 

Document $15 processing. 

Document $16 processing. 

Document $17 processing. 

Document $18 processing. 

Document $19 processing. 

Document $20 processing. 

Document $21 processing. 

Document $22 processing. 

Document $23 processing. 

Document $24 processing. 

Document $25 processing. 

Document $26 processing. 

Document $27 processing. 

Document $28 processing. 

Document $29 processing. 

Document $30 processing. 

Document $31 processing. 

Document $32 processing. 

Document $33 processing. 

Document $34 processing. 

Document $35 processing. 

Document $36 processing. 

Document $3

In [17]:
len(graph_documents)

130

In [25]:
print(len(graph_documents[0]))

1


In [32]:
print(graph_documents[0])

[GraphDocument(nodes=[Node(id='Ltimindtree', type='Organization', properties={}), Node(id='Amazon Web Services', type='Organization', properties={}), Node(id='Hartford', type='Location', properties={}), Node(id='Co-Innovation Center', type='Facility', properties={}), Node(id='Aws Competencies', type='Program', properties={}), Node(id='Service Delivery Program Designations', type='Program', properties={}), Node(id='Cloud Experts', type='Profession', properties={})], relationships=[Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Amazon Web Services', type='Organization', properties={}), type='PARTNER', properties={}), Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Hartford', type='Location', properties={}), type='HAS_FACILITY', properties={}), Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Co-Innovation Center', type='Facility', properties={}),

In [33]:
graph_documents_flat_list = []
for sublist in graph_documents:
    if sublist:  # Check if sublist is not empty
        graph_documents_flat_list.extend(sublist)

In [35]:
print(graph_documents_flat_list[1:3])

[GraphDocument(nodes=[Node(id='Ltimindtree', type='Organization', properties={}), Node(id='Aws', type='Organization', properties={}), Node(id='Certified Aws Professionals', type='Professionals', properties={}), Node(id='Application & Infrastructure', type='Service area', properties={})], relationships=[Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Aws', type='Organization', properties={}), type='PARTNERSHIP', properties={}), Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Certified Aws Professionals', type='Professionals', properties={}), type='HAS_CERTIFIED', properties={}), Relationship(source=Node(id='Ltimindtree', type='Organization', properties={}), target=Node(id='Application & Infrastructure', type='Service area', properties={}), type='OFFERS_SERVICES_IN', properties={})], source=Document(metadata={'source': 'finances.txt'}, page_content='and 1,855+ Certified AWS Professionals. LTIM

In [36]:
graph.add_graph_documents(
    graph_documents_flat_list,
    baseEntityLabel=True,
    include_source=True
)

In [37]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url="bolt://localhost:7688",          
    username="neo4j",                 
    password="F9gsV3zmLpQ2xT",
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

/var/folders/06/bmb8g_1d55q5lgskjgst444h0000gn/T/ipykernel_22584/3663268124.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
Python(24426) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
driver = GraphDatabase.driver(
        uri = "bolt://localhost:7688",
        auth = ("neo4j",
                "F9gsV3zmLpQ2xT"))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
# try:
# create_index()
# except:
#     pass

# Close the driver connection


driver.close()

In [39]:
class Entities(BaseModel):
    """Identifying information about financial or tech-related entities."""

    names: list[str] = Field(
        ...,
        description="All the person names, companies, or organizations that appear in the financial report or announcement.",
    )

# Updated prompt for financial context
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting companies, organizations, and key individuals from financial and technology partnership data.",
        ),
        (
            "human",
            "Extract all companies, organizations, and people mentioned in the following text: {question}",
        ),
    ]
)

entity_chain = llm.with_structured_output(Entities)

In [40]:
e = entity_chain.invoke("What are the key strategic partnerships LTIMindtree has formed with major cloud providers, and what services are offered through these collaborations?")

In [41]:
e

Entities(names=['LTIMindtree', 'Amazon Web Services', 'Microsoft Azure', 'Google Cloud Platform'])

In [62]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:5})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [43]:
print(graph_retriever("What are the key strategic partnerships LTIMindtree has formed with major cloud providers, and what services are offered through these collaborations?"))

Ltimindtree - PREMIER_TIER_PARTNER -> Amazon Web Services (Aws)
Ltimindtree - PARTNER -> Aws
Ltimindtree - PARTNER -> Cisco Meraki
Ltimindtree - PARTNER -> Cisco
Ltimindtree - PARTNER -> Databricks
Ltimindtree - PARTNER -> Pega
Ltimindtree - PARTNER -> Salesforce
Ltimindtree - PARTNER -> Servicenow
Ltimindtree - PARTNER -> Snowflake
Ltimindtree - PARTNER -> Oracle
Ltimindtree - PARTNER -> Public Sector Business
Ltimindtree - PARTNER -> Amazon Web Services
Ltimindtree - PARTNER -> Sap
Ltimindtree - PARTNER -> Game Changer Partner—Public Sector Business
Ltimindtree - PARTNER -> Google Cloud Platform
Ltimindtree - PARTNER -> Google Workspace
Ltimindtree - PARTNER -> Microsoft
Ltimindtree - PARTNER -> Azure
Ltimindtree - PARTNER -> Microsoft Ai
Ltimindtree - PARTNER -> Ibm
Ltimindtree - OFFERS -> Application
Ltimindtree - OFFERS -> Functional Consulting
Ltimindtree - OFFERS -> Integration Services
Ltimindtree - OFFERS -> Core Services
Ltimindtree - OFFERS -> Pega Launchpad
Ltimindtree - OF

In [56]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """

    return final_data

In [57]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

final_chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [63]:
print(graph_retriever("Which companies has LTIMindtree partnered with to drive innovations in Gen AI, and what are the unique contributions of each partner?"))

Microsoft - ANNOUNCED -> Ai Powered
Microsoft - SUPPORTED_BY -> Trained Associates
Microsoft - SUPPORTED_BY -> Certified Experts
Microsoft - SUPPORTS -> Ltimindtree
Ltimindtree - AWARDED_AS -> Microsoft
Ltimindtree - PARTNERSHIP -> Microsoft
Ltimindtree - PARTNER -> Microsoft
Ltimindtree - HAS_COMPETENCY -> Microsoft Workloads
Ltimindtree - PARTNER -> Microsoft Ai
Ltimindtree - MEMBER_OF -> Microsoft Partner Advisory Councils
Ltimindtree - USES -> Analytics On Microsoft Azure
Build And Modernize Ai Apps With Microsoft Azure - USES -> Analytics On Microsoft AzureLtimindtree - PARTNERS_WITH -> Aws
Ltimindtree - HAS_RELATIONSHIP -> Aws
Ltimindtree - PARTNERSHIP -> Aws
Company - PARTNER -> Aws
Ltimindtree - PARTNER -> Aws
Ltimindtree - HAS_COMPETENCY -> Aws Competencies
Aws Re:Invent - PARTNER_OF_THE_YEAR -> Ltimindtree
Aws Re:Invent - OCCURRING_AT -> November 2023
Aws Re:Invent - HOSTED_AT -> Las Vegas
Company - WORKS_ON -> Aws Fargate
Ltimindtree - DELIVERS -> Sap On AwsProfessional Serv

In [65]:
print(graph_retriever("Who all are LTIMindtree's elite partners"))

Ltimindtree - PREMIER_TIER_PARTNER -> Amazon Web Services (Aws)
Ltimindtree - PARTNER -> Aws
Ltimindtree - PARTNER -> Cisco Meraki
Ltimindtree - PARTNER -> Cisco
Ltimindtree - PARTNER -> Databricks
Ltimindtree - PARTNER -> Pega
Ltimindtree - PARTNER -> Salesforce
Ltimindtree - PARTNER -> Servicenow
Ltimindtree - PARTNER -> Snowflake
Ltimindtree - PARTNER -> Oracle
Ltimindtree - PARTNER -> Public Sector Business
Ltimindtree - PARTNER -> Amazon Web Services
Ltimindtree - PARTNER -> Sap
Ltimindtree - PARTNER -> Game Changer Partner—Public Sector Business
Ltimindtree - PARTNER -> Google Cloud Platform
Ltimindtree - PARTNER -> Google Workspace
Ltimindtree - PARTNER -> Microsoft
Ltimindtree - PARTNER -> Azure
Ltimindtree - PARTNER -> Microsoft Ai
Ltimindtree - PARTNER -> Ibm
Ltimindtree - OFFERS -> Application
Ltimindtree - OFFERS -> Functional Consulting
Ltimindtree - OFFERS -> Integration Services
Ltimindtree - OFFERS -> Core Services
Ltimindtree - OFFERS -> Pega Launchpad
Ltimindtree - OF

In [58]:
#Question 1:
print(final_chain.invoke(input="What are the key strategic partnerships LTIMindtree has formed with major cloud providers, and what services are offered through these collaborations?"))

LTIMindtree has formed key strategic partnerships with major cloud providers including Amazon Web Services (AWS), Microsoft Azure, Google Cloud Platform, and Oracle. Through these collaborations, LTIMindtree offers a wide range of services such as application development, functional consulting, integration services, managed IT services, and cloud solutions, aimed at maximizing ROI and delivering innovative solutions to clients.


In [ ]:
#Question 2
print(final_chain.invoke(input="What does LTIMindtree offer to its other partners?"))

LTIMindtree offers a range of services to its partners, including consulting and implementation, managed IT services, integration services, and industry cloud solutions. Additionally, it provides specialized offerings like Pega Launchpad, Oracle Cloud applications, and various technology platforms, all aimed at improving business outcomes and delivering personalized customer experiences.


In [67]:
#Question 2
print(final_chain.invoke(input="What does LTIMindtree offers to Databricks, servicenow, oracle and dell?"))

LTIMindtree offers a range of services and solutions to its partners:

- **Databricks**: LTIMindtree enables customers to leverage the Databricks Lakehouse Platform and has a dedicated Databricks Business Unit to enhance joint go-to-market strategies. It also has a strong Databricks Practice with over 1,500 trained resources.

- **ServiceNow**: LTIMindtree develops ServiceNow solutions and invests in key markets to strengthen its delivery capabilities, enhancing its overall footprint in the ServiceNow ecosystem.

- **Oracle**: LTIMindtree provides Oracle Cloud services and has developed expertise in Oracle solutions, collaborating closely with Oracle to deliver comprehensive offerings.

- **Dell**: LTIMindtree partners with Dell Technologies to provide services that combine Dell's infrastructure and software with AI capabilities, driving transformative strategies for clients.


In [68]:
#Question 2
print(final_chain.invoke(input="Describe LTIMindTree's relationship with IBM."))

LTIMindtree has formed a strong partnership with IBM, characterized by deep synergy that fosters innovation and excellence. This collaboration includes joint efforts in various areas, such as engaging with IBM Quantum and participating in the IBM Partner Plus Award program. LTIMindtree's relationship with IBM is strategic, enhancing their capabilities and offerings to clients, particularly in the realms of digital transformation and cloud solutions.


In [70]:
print(graph_retriever("What global recognitions has LTIMindtree received from its partners, and what do they signify about its capabilities?"))

Ltimindtree - PREMIER_TIER_PARTNER -> Amazon Web Services (Aws)
Ltimindtree - PARTNER -> Aws
Ltimindtree - PARTNER -> Cisco Meraki
Ltimindtree - PARTNER -> Cisco
Ltimindtree - PARTNER -> Databricks
Ltimindtree - PARTNER -> Pega
Ltimindtree - PARTNER -> Salesforce
Ltimindtree - PARTNER -> Servicenow
Ltimindtree - PARTNER -> Snowflake
Ltimindtree - PARTNER -> Oracle
Ltimindtree - PARTNER -> Public Sector Business
Ltimindtree - PARTNER -> Amazon Web Services
Ltimindtree - PARTNER -> Sap
Ltimindtree - PARTNER -> Game Changer Partner—Public Sector Business
Ltimindtree - PARTNER -> Google Cloud Platform
Ltimindtree - PARTNER -> Google Workspace
Ltimindtree - PARTNER -> Microsoft
Ltimindtree - PARTNER -> Azure
Ltimindtree - PARTNER -> Microsoft Ai
Ltimindtree - PARTNER -> Ibm
Ltimindtree - OFFERS -> Application
Ltimindtree - OFFERS -> Functional Consulting
Ltimindtree - OFFERS -> Integration Services
Ltimindtree - OFFERS -> Core Services
Ltimindtree - OFFERS -> Pega Launchpad
Ltimindtree - OF

In [71]:
print(final_chain.invoke(input="Compare LTIMindtree’s approach to joint go-to-market (GTM) strategies across at least two of its major partnerships."))

LTIMindtree employs a comprehensive 360-degree partnership approach in its joint go-to-market (GTM) strategies, which is evident in its collaborations with major partners like Amazon Web Services (AWS) and Microsoft.

With AWS, LTIMindtree focuses on a robust partnership that includes competencies in areas such as IoT, Data & Analytics, and Machine Learning. This partnership is highlighted by their recognition as AWS Partner of the Year at AWS Re:Invent, showcasing their commitment to delivering innovative solutions on the AWS platform. Their GTM strategy involves leveraging AWS's cloud capabilities to enhance service offerings, particularly in managed IT services and industry-specific solutions.

In contrast, LTIMindtree's partnership with Microsoft emphasizes a collaborative ecosystem that supports various Microsoft technologies, including Azure and Microsoft AI. The partnership is reinforced by LTIMindtree's membership in Microsoft Partner Advisory Councils and its use of analytics 

In [72]:
print(final_chain.invoke(input="Who all are LTIMindtree's premier sponsor or premier partners?"))

LTIMindtree's premier partners include Amazon Web Services (AWS) and SAP, as well as its status as a Global Elite partner of ServiceNow and Pega.


In [73]:
print(final_chain.invoke(input="Which partner awards or recognitions received by LTIMindtree reflect a deep business alignment and joint innovation focus, and how do these accolades differ across partners?"))

LTIMindtree has received several awards and recognitions that reflect its deep business alignment and joint innovation focus with partners. Notably, it was awarded the "Partner of the Year" at AWS Re:Invent, highlighting its strong collaboration with Amazon Web Services. Additionally, LTIMindtree's partnership with IBM has fostered innovation and excellence, showcasing a 360-degree relationship that emphasizes joint efforts in creativity and service delivery.

These accolades differ across partners in their specific focus areas: the AWS recognition underscores cloud services and infrastructure capabilities, while the IBM partnership emphasizes hybrid cloud solutions and innovation. Each award reflects the unique strengths and collaborative efforts LTIMindtree has with its partners, tailored to their respective domains.
